Импортируем необходимые библиотеки

In [2]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
from matplotlib.image import imread
from mpl_toolkits import mplot3d
from matplotlib import gridspec
import seaborn as sns
import pandas as pd
from tqdm.notebook import tqdm

from scipy.special import softmax
from scipy.spatial.distance import cdist
import numpy as np
import torch

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold, ParameterGrid, GridSearchCV, LeaveOneOut, RepeatedKFold
from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier


from sklearn.decomposition import PCA

from sklearn.datasets import load_breast_cancer

#from mlxtend.plotting import plot_decision_regions

#from torchvision import datasets
#from torchvision import transforms

Загружаем данные

In [3]:
train_data = pd.read_csv('train.csv', index_col=0)

In [4]:
train_data.head()

,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,...,x_292,x_293,x_294,x_295,x_296,x_297,x_298,x_299,x_300,Category
Id,,,,,,,,,,,,,,,,,,,,,
0,-0.03,-1.00,-0.02,-0.80,-1.37,0.62,-0.09,-0.87,-0.02,0.63,...,-0.16,-0.02,-0.15,-0.00,1.07,-0.06,-0.80,0.39,-0.06,-1.0
1,-0.18,1.71,0.00,-1.09,-1.13,-0.08,2.20,-0.32,0.00,0.81,...,-0.63,0.08,-1.83,0.04,1.49,0.10,0.78,-1.56,0.08,1.0
2,-1.41,0.01,0.00,-0.48,0.31,1.02,0.90,0.41,0.04,-0.00,...,-1.25,0.06,-0.20,0.04,-0.77,0.04,-1.39,0.87,0.02,1.0
3,-0.41,-0.54,-0.02,-0.53,0.52,-1.23,0.62,0.02,-0.03,0.06,...,0.49,-0.00,-0.40,-0.02,-1.02,-0.00,-0.16,0.41,-0.07,-1.0
4,-1.58,0.25,0.02,1.15,-1.95,-0.43,0.40,-0.33,0.02,0.72,...,-0.14,0.04,-0.85,0.00,0.02,0.14,0.68,-1.42,0.02,-1.0


In [5]:
X = train_data.iloc[:, train_data.columns != 'Category'].values
y = train_data.iloc[:, train_data.columns == 'Category'].values.reshape(-1)

In [6]:
y[y==-1]=0 #change category encoding

Научим перцептрон

In [35]:

hidden_layers = []
for i in (1,2):
  hidden_layers.append(tuple(np.full(i, 64)))
  hidden_layers.append(tuple(np.full(i, 8)))


In [ ]:
perceptron_search = GridSearchCV(MLPClassifier(), {'hidden_layer_sizes': hidden_layers, 'activation': ["relu"], 'random_state':[0], 'max_iter': [2000]}, scoring="accuracy", verbose = 5)
perceptron_search.fit(X, y)

In [ ]:
print("Best params:", perceptron_search.best_params_)
print("Accuracy:", perceptron_search.best_score_)

Но сначала проведем небольшую обработку данных

In [58]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_rescaled = scaler.fit_transform(X)


In [59]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.99)
pca.fit(data_rescaled)
reduced = pca.transform(data_rescaled)

In [61]:
reduced

array([[-3.27372848e-01, -4.47545680e-06,  1.20260708e-01, ...,
        -3.97655529e-02,  1.16275670e-02,  3.28827140e-02],
       [ 2.98662906e-01, -1.87822400e-01,  5.78835404e-02, ...,
         8.75803372e-03, -3.61526443e-02, -1.38153353e-02],
       [ 2.96162414e-01,  1.63361488e-01,  1.07054043e-01, ...,
         4.49794840e-03, -1.66346296e-02, -1.67063034e-02],
       ...,
       [-1.51131299e-01,  2.28502915e-01, -1.63792861e-01, ...,
         2.19389525e-03,  2.02563023e-02,  1.39219152e-02],
       [ 2.95691425e-01,  8.36946449e-02, -1.03082755e-01, ...,
         3.64330964e-02, -4.55534308e-02,  4.76137454e-02],
       [ 1.52526338e-01,  7.32562773e-02,  9.64641328e-02, ...,
         2.67719813e-02,  9.76538935e-03,  2.20303937e-02]])

In [62]:
Perceptron_best_model = MLPClassifier(hidden_layer_sizes= (64,64), activation = "relu", random_state = 0, max_iter = 2000)

X_train, X_test, y_train, y_test = train_test_split(reduced, y, test_size=0.2)
Perceptron_best_model.fit(X_train, y_train )
Perceptron_best_report = classification_report(Perceptron_best_model.predict(X_test), y_test)
print("Perceptron with (64, 64) hidden layers\n", Perceptron_best_report)

Perceptron with (64, 64) hidden layers
               precision    recall  f1-score   support

         0.0       0.96      0.97      0.97      8965
         1.0       0.97      0.96      0.97      9035

    accuracy                           0.97     18000
   macro avg       0.97      0.97      0.97     18000
weighted avg       0.97      0.97      0.97     18000



In [63]:
Perceptron_best_model.fit(reduced, y)

MLPClassifier(hidden_layer_sizes=(64, 64), max_iter=2000, random_state=0)

In [43]:
test_data = pd.read_csv('test.csv', index_col=0)

In [44]:
X_test_data = test_data.iloc[:, test_data.columns != 'Category'].values

In [67]:
X_rescaled = scaler.fit_transform(X_test_data)
X_pca = pca.transform(X_rescaled)

predict = Perceptron_best_model.predict(X_pca)
predict[predict == 0] = -1

In [70]:
outcome = pd.DataFrame(predict).astype(int)
pass_id = pd.read_csv('test.csv')[['Id']]
result = pd.concat([pass_id,outcome], axis=1)
result.columns = ['Id','Category']
result.to_csv('result_new format.csv',encoding='utf-8', columns=['Id','Category'], index=False)

In [75]:
result.head()

,Id,Category
0,0,-1
1,1,-1
2,2,1
3,3,-1
4,4,-1


Итак, результат с PCA хуже чем без него. Оставим предыдущий результат с двуслойной сеткой.

In [76]:
Perceptron_best_model.fit(X, y)

MLPClassifier(hidden_layer_sizes=(64, 64), max_iter=2000, random_state=0)

In [77]:
predict = Perceptron_best_model.predict(X_test_data)
predict[predict == 0] = -1

In [78]:
outcome = pd.DataFrame(predict).astype(int)
pass_id = pd.read_csv('test.csv')[['Id']]
result = pd.concat([pass_id,outcome], axis=1)
result.columns = ['Id','Category']
result.to_csv('result_without PCA.csv',encoding='utf-8', columns=['Id','Category'], index=False)